In [1]:
from np_struct import Packet, Struct
from np_struct.transfer import SocketInterface, PacketServer
import numpy as np

## Structured Arrays

Struct members are class variables, and can be other `Struct` objects, numpy arrays, or one of the standard numpy data types.
When creating a new struct, any member can be initialized by passing in an initial value.

In [2]:
class pktheader(Struct):
    psize = np.uint16()
    ptype = np.uint8()
    payload_shape = np.uint8([1, 1])

pkt = pktheader(ptype=5)
pkt

Struct pktheader: 
    psize:          uint16[0]
    ptype:          uint8[5]
    payload_shape:  uint8[1 1]

`Struct` members will be broadcasted to the shape of the initial value.

In [3]:
pkt = pktheader(ptype=np.arange(5))
pkt

Struct pktheader: 
    psize:          uint16[0]
    ptype:          uint8[0 1 2 3 4]
    payload_shape:  uint8[1 1]

An array of structs can be created by passing in a `shape`.

In [4]:
pkt_array = pktheader(ptype=5, shape=(3,2))
pkt_array

Struct pktheader: (3, 2)
[
    psize:          uint16[0]
    ptype:          uint8[5]
    payload_shape:  uint8[1 1]
]
...
[
    psize:          uint16[0]
    ptype:          uint8[5]
    payload_shape:  uint8[1 1]
]

Structures can also include other structures.

In [5]:
class expkt(Struct):
    hdr = pktheader(ptype=0x2)
    da = np.arange(8)

ex = expkt()
ex

Struct expkt: 
    hdr:  Struct pktheader: 
              psize:          uint16[0]
              ptype:          uint8[2]
              payload_shape:  uint8[1 1]
    da:   int64[0 1 2 3 4 5 6 7]

## Accessing and Setting Members

Members can be accessed either with indexing, or using the dot operator.

In [6]:
ex.da *= 2
ex['da']

array([ 0,  2,  4,  6,  8, 10, 12, 14])

In [7]:
pkt_array[0,0].ptype = 15
pkt_array

Struct pktheader: (3, 2)
[
    psize:          uint16[0]
    ptype:          uint8[15]
    payload_shape:  uint8[1 1]
]
...
[
    psize:          uint16[0]
    ptype:          uint8[5]
    payload_shape:  uint8[1 1]
]

## Writing to Disk

In [8]:
np.save('test.npy', pkt_array)
pktheader(np.load('test.npy'))

Struct pktheader: (3, 2)
[
    psize:          uint16[0]
    ptype:          uint8[15]
    payload_shape:  uint8[1 1]
]
...
[
    psize:          uint16[0]
    ptype:          uint8[5]
    payload_shape:  uint8[1 1]
]

## Interfaces

To send structured arrays across a serial or socket interface, the structure must inherit from `Packet` and include a header that contains a type field that is unique for each packet sub-class that will be sent across the interface. 

The header must implement the `get_ptype` method that returns the unique identifier field.

In [9]:
from np_struct.bitfields import uint16

class pktheader(Packet):
    psize = np.uint16()
    ptype = np.uint8()
    payload_shape = np.uint8([1, 1])
    
    def get_ptype(self):
        return self.ptype
    
class datapkt(Packet):
    hdr = pktheader(ptype=0x2)
    da = np.uint16()

class cmdpkt(Packet):
    hdr = pktheader(ptype=0x03)
    state1 = uint16(bits=7)
    state2 = uint16(bits=3)
    state3 = uint16(bits=1)

class ack(Packet):
    hdr = pktheader(ptype=0xFF)
    ack_type = np.uint8()
    ack_code = np.uint8()

class variablepkt(Packet):
    hdr = pktheader(ptype=0x0A)
    da = np.uint16()

    @classmethod
    def from_header(cls, hdr: pktheader, **kwargs):
        return cls(da=np.zeros(hdr.payload_shape), **kwargs)

To implement an interface that can parse these packets, pass in the header class to the interface constructor,

In [10]:
client_intf = SocketInterface(target=('localhost', 50010), header=pktheader)

Create a simple server that manipulates the packets sent by the client,

In [11]:
def pkt_handler(pkt: Packet) -> Packet:
    """
    Server-side packet handler. Given a packet from the client, create a new packet to send back.
    """
    # modify packet data
    if isinstance(pkt, datapkt):
        pkt.da *= 2
        return pkt
    # send acknowledgement packet with error code
    elif isinstance(pkt, cmdpkt):
        txpkt = ack(ack_code=pkt.state2)
        txpkt.ack_type = pkt.hdr.ptype
        return txpkt
    # change the size of the returned packet
    elif isinstance(pkt, variablepkt):
        txpkt = variablepkt(da=np.arange(16))
        txpkt.hdr.payload_shape = [1, 16]
        return txpkt
    # loopback to client
    else:
        raise ValueError(f"Unsupported packet type: {pkt.__class__.__name__}")

server = PacketServer(
    host=('localhost', 50010), header=pktheader, pkt_handler=pkt_handler, timeout=0.02
)

# start server
server.start()

Test out the server with a a few different packet types,

In [12]:
# returned data from the server should be populated with random integers
ex = datapkt(da = 3)

with client_intf as client:
    rxpkt = client.pkt_sendrecv(ex)

rxpkt.da


array([6], dtype=uint16)

In [13]:
# the ack_type member in the returned data should be populated with the ptype field of the sent packet
cmd = cmdpkt()
cmd.state2 = 0x4
with client_intf as client:
    rxpkt = client.pkt_sendrecv(cmd)

print(cmd)
rxpkt

Packet cmdpkt: 
    hdr:          Packet pktheader: 
                      psize:          uint16[0]
                      ptype:          uint8[3]
                      payload_shape:  uint8[1 1]
    state1:       uint16(7:0)[0]
    state2:       uint16(10:7)[4]
    state3:       uint16(11:10)[0]


Packet ack: 
    hdr:       Packet pktheader: 
                   psize:          uint16[0]
                   ptype:          uint8[255]
                   payload_shape:  uint8[1 1]
    ack_type:  uint8[3]
    ack_code:  uint8[4]

## Variable Length Packets

Struct members can be initialized with any shape, and the same is true of packets. When the interface is receiving and parsing a packet, it will call the `from_header` method if it is defined in the `Packet` class. The received bytes will be unpacked into the empty packet returned by `from_header`. This allows the packet members to be defined with variable shapes.

The `variablepkt` class implemented `from_header` and returns a new packet with the `da` field initialized to the shape
in the `payload_shape` header field. This tells the packet reader what size to expect for the full packet by only reading the header.

In [14]:
with client_intf as client:
    data = np.arange(6).reshape(2, 3)
    v = variablepkt(da=data)
    v.hdr.payload_shape = data.shape
    rxpkt = client.pkt_sendrecv(v)

print(v)
rxpkt

Packet variablepkt: 
    hdr:  Packet pktheader: 
              psize:          uint16[0]
              ptype:          uint8[10]
              payload_shape:  uint8[2 3]
    da:   uint16[[0 1 2]
	        [3 4 5]]


Packet variablepkt: 
    hdr:  Packet pktheader: 
              psize:          uint16[0]
              ptype:          uint8[10]
              payload_shape:  uint8[ 1 16]
    da:   uint16[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]]

In [15]:
server.stop()